In [ ]:
# ============================================================
# STEP 1 — Google Drive + Imports
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# STEP 2 — Load SPEED-only Normalized Data
# ============================================================
data_dir = "/content/drive/MyDrive/traffic_data"

train_full = np.load(f"{data_dir}/train_speed_norm.npz")["data"]  # (36481, 325)
val_full   = np.load(f"{data_dir}/val_speed_norm.npz")["data"]    # (5211, 325)
test_full  = np.load(f"{data_dir}/test_speed_norm.npz")["data"]   # (10424, 325)

print("Loaded shapes:", train_full.shape, val_full.shape, test_full.shape)

train_data = train_full
val_data   = val_full
test_data  = test_full

print("Train shape:", train_data.shape)
print("Val shape:",   val_data.shape)
print("Test shape:",  test_data.shape)

# ============================================================
# STEP 3 — Load REAL 325×325 Adjacency Matrix
# ============================================================
adj_path = f"{data_dir}/adj_mx_bay.pkl/adj_mx_bay.pkl"

with open(adj_path, "rb") as f:
    sensor_ids, sensor_id_to_ind, adj_mx = pickle.load(f, encoding="latin1")

adj_mx = np.array(adj_mx).astype(np.float32)
NUM_NODES = adj_mx.shape[0]

print("Adjacency shape:", adj_mx.shape)   # (325, 325)

# ============================================================
# STEP 4 — Build Temporal Sequences (speed-only)
# ============================================================
def create_sequences(data, seq_len=12, pred_len=3):
    """
    data: (T, N)
    returns:
       X: (B, seq_len, N, 1)
       Y: (B, pred_len, N, 1)
    """
    T = data.shape[0]
    X, Y = [], []

    for i in range(T - seq_len - pred_len):
        seq_x = data[i:i+seq_len]                  # (seq_len, 325)
        seq_y = data[i+seq_len:i+seq_len+pred_len] # (pred_len, 325)

        X.append(seq_x[..., None])  # Add feature dim → (seq_len, 325, 1)
        Y.append(seq_y[..., None])

    return np.array(X), np.array(Y)

# Build sequences
seq_len = 12
pred_len = 3

X_train, Y_train = create_sequences(train_data, seq_len, pred_len)
X_val, Y_val     = create_sequences(val_data, seq_len, pred_len)
X_test, Y_test   = create_sequences(test_data, seq_len, pred_len)

print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)

# ============================================================
# STEP 5 — DataLoaders
# ============================================================
def to_loader(X, Y, batch_size=32, shuffle=True):
    return DataLoader(
        TensorDataset(torch.FloatTensor(X), torch.FloatTensor(Y)),
        batch_size=batch_size,
        shuffle=shuffle
    )

train_loader = to_loader(X_train, Y_train)
val_loader   = to_loader(X_val, Y_val)
test_loader  = to_loader(X_test, Y_test)

# ============================================================
# STEP 6 — MULTI-LAYER DCRNN MODEL (5 layers × 128 hidden)
# ============================================================
class DiffusionConv(nn.Module):
    def __init__(self, num_nodes, input_dim, output_dim, dropout=0.2):
        super().__init__()
        self.theta = nn.Parameter(torch.randn(input_dim, output_dim))
        nn.init.xavier_uniform_(self.theta)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, adj):
        out = torch.einsum("ij,bjf->bif", adj, x)
        out = torch.einsum("bif,fo->bio", out, self.theta)
        return self.dropout(out)


class DCRNNCell(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, dropout=0.2):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim

        self.diff = DiffusionConv(num_nodes, input_dim + hidden_dim, hidden_dim, dropout)
        self.gru = nn.GRUCell(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, h_prev, adj):
        combined = torch.cat([x, h_prev], dim=-1)
        diff_out = self.dropout(self.diff(combined, adj))
        h_new = self.gru(diff_out.reshape(-1, self.hidden_dim),
                         h_prev.reshape(-1, self.hidden_dim))
        return h_new.reshape(-1, self.num_nodes, self.hidden_dim)


class DCRNN(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, output_dim, adj, num_layers=5, dropout=0.2):
        super().__init__()

        self.layers = nn.ModuleList()
        self.layers.append(DCRNNCell(num_nodes, input_dim, hidden_dim, dropout))
        for _ in range(num_layers - 1):
            self.layers.append(DCRNNCell(num_nodes, hidden_dim, hidden_dim, dropout))

        self.fc = nn.Linear(hidden_dim, output_dim)
        self.register_buffer("adj", torch.FloatTensor(adj))

    def forward(self, x):
        B, T, N, F = x.shape
        h = [torch.zeros(B, N, hidden_dim, device=x.device) for _ in range(len(self.layers))]

        for t in range(T):
            input_t = x[:, t]   # (B, 325, 1)
            for l, layer in enumerate(self.layers):
                h[l] = layer(input_t, h[l], self.adj)
                input_t = h[l]  # Pass output to next layer

        out = self.fc(input_t)    # (B, 325, 1)
        return out.unsqueeze(1)   # (B, 1, 325, 1)

# ============================================================
# STEP 7 — TRAINING (with best model saving)
# ============================================================
input_dim = 1
hidden_dim = 128
output_dim = 1

model = DCRNN(NUM_NODES, input_dim, hidden_dim, output_dim, adj_mx, num_layers=5, dropout=0.2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

EPOCHS = 50
best_val_loss = float("inf")
best_model_path = f"{data_dir}/dcrnn_speed_best.pth"

for epoch in range(EPOCHS):

    model.train()
    train_loss = 0.0

    for Xb, Yb in train_loader:
        Xb, Yb = Xb.to(device), Yb.to(device)

        optimizer.zero_grad()
        preds = model(Xb)                 # (B, 1, 325, 1)
        target = Yb[:, -1:]               # final step (B, 1, 325, 1)

        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for Xb, Yb in val_loader:
            preds = model(Xb.to(device))
            target = Yb[:, -1:].to(device)
            val_loss += criterion(preds, target).item()

    avg_train = train_loss / len(train_loader)
    avg_val = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}  Train={avg_train:.6f}  Val={avg_val:.6f}")

    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print(f"🔥 Best model saved (Val={best_val_loss:.6f}) → {best_model_path}")

print("\nTraining complete!")
print(f"Best model saved at: {best_model_path}")


Mounted at /content/drive
Using device: cuda
Loaded shapes: (36481, 325) (5211, 325) (10424, 325)
Train shape: (36481, 325)
Val shape: (5211, 325)
Test shape: (10424, 325)
Adjacency shape: (325, 325)
X_train: (36466, 12, 325, 1)
Y_train: (36466, 3, 325, 1)
Epoch 1/50  Train=0.426151  Val=0.436007
🔥 Best model saved (Val=0.436007) → /content/drive/MyDrive/traffic_data/dcrnn_speed_best.pth


128 - Hidden Dimension

In [ ]:
# ============================================================
# STEP 1 — Google Drive + Imports
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# STEP 2 — Load Normalized Data
# ============================================================
data_dir = "/content/drive/MyDrive/traffic_data"

train_full = np.load(f"{data_dir}/train_norm_PEMS_BAY.npz")["data"]
val_full   = np.load(f"{data_dir}/val_norm_PEMS_BAY.npz")["data"]
test_full  = np.load(f"{data_dir}/test_norm_PEMS_BAY.npz")["data"]

print("Loaded shapes:", train_full.shape, val_full.shape, test_full.shape)

# Limit timesteps EXACTLY as requested
train_data = train_full[:36481]   # (1000, 325, 3)
val_data   = val_full[:5211]      # (100, 325, 3)
test_data  = test_full[:10424]     # (200, 325, 3)

print("Train shape:", train_data.shape)
print("Val shape:",   val_data.shape)
print("Test shape:",  test_data.shape)

# ============================================================
# STEP 3 — Load REAL 325×325 Adjacency Matrix
# ============================================================
adj_path = f"{data_dir}/adj_mx_bay.pkl/adj_mx_bay.pkl"

with open(adj_path, "rb") as f:
    sensor_ids, sensor_id_to_ind, adj_mx = pickle.load(f, encoding="latin1")

adj_mx = np.array(adj_mx).astype(np.float32)
NUM_NODES = adj_mx.shape[0]

print("Adjacency shape:", adj_mx.shape)   # (325, 325)


# ============================================================
# STEP 4 — Build Temporal Sequences
# ============================================================
def create_sequences(data, seq_len=12, pred_len=3):
    """
    data: (T, N, F)
    returns:
       X: (B, seq_len, N, F)
       Y: (B, pred_len, N, F)
    """
    T = data.shape[0]
    X, Y = [], []

    for i in range(T - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        Y.append(data[i+seq_len:i+seq_len+pred_len])

    X = np.array(X)
    Y = np.array(Y)
    return X, Y

# Build sequences
seq_len = 12
pred_len = 3

X_train, Y_train = create_sequences(train_data, seq_len, pred_len)
X_val, Y_val     = create_sequences(val_data, seq_len, pred_len)
X_test, Y_test   = create_sequences(test_data, seq_len, pred_len)

print("Sequence shapes:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)


# ============================================================
# STEP 5 — DataLoaders
# ============================================================
def to_loader(X, Y, batch=32, shuffle=True):
    return DataLoader(
        TensorDataset(torch.FloatTensor(X), torch.FloatTensor(Y)),
        batch_size=batch,
        shuffle=shuffle
    )

train_loader = to_loader(X_train, Y_train)
val_loader   = to_loader(X_val, Y_val)
test_loader  = to_loader(X_test, Y_test)


# ============================================================
# STEP 6 — DCRNN MODEL
# ============================================================
# ============================================================
# STEP 6 — DCRNN MODEL (with dropout=0.2)
# ============================================================
# ============================================================
# STEP 6 — Multilayer DCRNN (5 layers, 128 units each, dropout=0.2)
# ============================================================
class DiffusionConv(nn.Module):
    def __init__(self, num_nodes, input_dim, output_dim, dropout=0.2):
        super().__init__()
        self.theta = nn.Parameter(torch.randn(input_dim, output_dim))
        nn.init.xavier_uniform_(self.theta)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, adj):
        # x: (B, N, F)
        out = torch.einsum("ij,bjf->bif", adj, x)        # diffusion step
        out = torch.einsum("bif,fo->bio", out, self.theta)
        out = self.dropout(out)                          # dropout
        return out


class DCRNNCell(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, dropout=0.2):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim

        self.diff = DiffusionConv(num_nodes, input_dim + hidden_dim, hidden_dim, dropout)
        self.gru = nn.GRUCell(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, h_prev, adj):
        combined = torch.cat([x, h_prev], dim=-1)
        diff_out = self.diff(combined, adj)
        diff_out = self.dropout(diff_out)
        h_new = self.gru(diff_out.reshape(-1, self.hidden_dim),
                         h_prev.reshape(-1, self.hidden_dim))
        return h_new.reshape(-1, self.num_nodes, self.hidden_dim)


class DCRNN(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, output_dim, adj, dropout=0.2, num_layers=5):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # -------- Create 5 stacked DCRNN layers --------
        layers = []
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            layers.append(DCRNNCell(num_nodes, in_dim, hidden_dim, dropout))

        self.layers = nn.ModuleList(layers)

        # Final output projection
        self.fc_dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.register_buffer("adj", torch.FloatTensor(adj))

    def forward(self, x):
        B, T, N, F = x.shape
        h = [torch.zeros(B, N, self.hidden_dim, device=x.device)
             for _ in range(self.num_layers)]

        for t in range(T):
            input_t = x[:, t]   # (B, N, F)
            for l in range(self.num_layers):
                h[l] = self.layers[l](input_t, h[l], self.adj)
                input_t = h[l]   # output of one layer becomes input to next

        out = self.fc_dropout(h[-1])
        out = self.fc(out)
        return out.unsqueeze(1)




# ============================================================
# STEP 7 — TRAIN DCRNN
# ============================================================
# ============================================================
# STEP 7 — TRAIN DCRNN (Save best validation model)
# ============================================================
input_dim = 3
hidden_dim = 128
output_dim = 1

model = DCRNN(NUM_NODES, input_dim, hidden_dim, output_dim, adj_mx, dropout=0.2, num_layers=5).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

EPOCHS = 50
best_val_loss = float('inf')    # Track best loss

best_model_path = f"{data_dir}/dcrnn_325nodes_best.pth"

for epoch in range(EPOCHS):
    # ------------------------
    # TRAINING
    # ------------------------
    model.train()
    train_loss = 0

    for Xb, Yb in train_loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        optimizer.zero_grad()

        preds = model(Xb)               # (B, 1, N, 1)
        target = Yb[:, -1:, :, 0:1]     # final step target

        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ------------------------
    # VALIDATION
    # ------------------------
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for Xb, Yb in val_loader:
            preds = model(Xb.to(device))
            target = Yb[:, -1:, :, 0:1].to(device)
            val_loss += criterion(preds, target).item()

    avg_train = train_loss / len(train_loader)
    avg_val   = val_loss   / len(val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}  Train={avg_train:.6f}  Val={avg_val:.6f}")

    # ------------------------
    # SAVE BEST MODEL
    # ------------------------
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print(f"🔥 Saved BEST model (Val loss = {best_val_loss:.6f}) at: {best_model_path}")

# ============================================================
# FINAL PRINT
# ============================================================
print(f"\nTraining done. Best model saved at:\n → {best_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Loaded shapes: (36481, 325, 3) (5211, 325, 3) (10424, 325, 3)
Train shape: (36481, 325, 3)
Val shape: (5211, 325, 3)
Test shape: (10424, 325, 3)
Adjacency shape: (325, 325)
Sequence shapes:
X_train: (36466, 12, 325, 3)
Y_train: (36466, 3, 325, 3)
Epoch 1/50  Train=0.256532  Val=0.274483
🔥 Saved BEST model (Val loss = 0.274483) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 2/50  Train=0.225064  Val=0.264221
🔥 Saved BEST model (Val loss = 0.264221) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 3/50  Train=0.220301  Val=0.263189
🔥 Saved BEST model (Val loss = 0.263189) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 4/50  Train=0.217802  Val=0.258601
🔥 Saved BEST model (Val loss = 0.258601) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 5/50  Train=0.2

Multilayer DCRNN (3 layers, 64 units each, dropout=0.2)


In [ ]:
# ============================================================
# STEP 1 — Google Drive + Imports
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# STEP 2 — Load Normalized Data
# ============================================================
data_dir = "/content/drive/MyDrive/traffic_data"

train_full = np.load(f"{data_dir}/train_norm_PEMS_BAY.npz")["data"]
val_full   = np.load(f"{data_dir}/val_norm_PEMS_BAY.npz")["data"]
test_full  = np.load(f"{data_dir}/test_norm_PEMS_BAY.npz")["data"]

print("Loaded shapes:", train_full.shape, val_full.shape, test_full.shape)

# Limit timesteps EXACTLY as requested
train_data = train_full[:36481]   # (1000, 325, 3)
val_data   = val_full[:5211]      # (100, 325, 3)
test_data  = test_full[:10424]     # (200, 325, 3)

print("Train shape:", train_data.shape)
print("Val shape:",   val_data.shape)
print("Test shape:",  test_data.shape)

# ============================================================
# STEP 3 — Load REAL 325×325 Adjacency Matrix
# ============================================================
adj_path = f"{data_dir}/adj_mx_bay.pkl/adj_mx_bay.pkl"

with open(adj_path, "rb") as f:
    sensor_ids, sensor_id_to_ind, adj_mx = pickle.load(f, encoding="latin1")

adj_mx = np.array(adj_mx).astype(np.float32)
NUM_NODES = adj_mx.shape[0]

print("Adjacency shape:", adj_mx.shape)   # (325, 325)


# ============================================================
# STEP 4 — Build Temporal Sequences
# ============================================================
def create_sequences(data, seq_len=12, pred_len=3):
    """
    data: (T, N, F)
    returns:
       X: (B, seq_len, N, F)
       Y: (B, pred_len, N, F)
    """
    T = data.shape[0]
    X, Y = [], []

    for i in range(T - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        Y.append(data[i+seq_len:i+seq_len+pred_len])

    X = np.array(X)
    Y = np.array(Y)
    return X, Y

# Build sequences
seq_len = 12
pred_len = 3

X_train, Y_train = create_sequences(train_data, seq_len, pred_len)
X_val, Y_val     = create_sequences(val_data, seq_len, pred_len)
X_test, Y_test   = create_sequences(test_data, seq_len, pred_len)

print("Sequence shapes:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)


# ============================================================
# STEP 5 — DataLoaders
# ============================================================
def to_loader(X, Y, batch=32, shuffle=True):
    return DataLoader(
        TensorDataset(torch.FloatTensor(X), torch.FloatTensor(Y)),
        batch_size=batch,
        shuffle=shuffle
    )

train_loader = to_loader(X_train, Y_train)
val_loader   = to_loader(X_val, Y_val)
test_loader  = to_loader(X_test, Y_test)


# ============================================================
# STEP 6 — DCRNN MODEL
# ============================================================
# ============================================================
# STEP 6 — DCRNN MODEL (with dropout=0.2)
# ============================================================
# ============================================================
# STEP 6 — Multilayer DCRNN (3 layers, 64 units each, dropout=0.2)
# ============================================================
class DiffusionConv(nn.Module):
    def __init__(self, num_nodes, input_dim, output_dim, dropout=0.2):
        super().__init__()
        self.theta = nn.Parameter(torch.randn(input_dim, output_dim))
        nn.init.xavier_uniform_(self.theta)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, adj):
        # x: (B, N, F)
        out = torch.einsum("ij,bjf->bif", adj, x)        # diffusion step
        out = torch.einsum("bif,fo->bio", out, self.theta)
        out = self.dropout(out)                          # dropout
        return out


class DCRNNCell(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, dropout=0.2):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim

        self.diff = DiffusionConv(num_nodes, input_dim + hidden_dim, hidden_dim, dropout)
        self.gru = nn.GRUCell(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, h_prev, adj):
        combined = torch.cat([x, h_prev], dim=-1)
        diff_out = self.diff(combined, adj)
        diff_out = self.dropout(diff_out)
        h_new = self.gru(diff_out.reshape(-1, self.hidden_dim),
                         h_prev.reshape(-1, self.hidden_dim))
        return h_new.reshape(-1, self.num_nodes, self.hidden_dim)


class DCRNN(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, output_dim, adj, dropout=0.2, num_layers=5):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # -------- Create 5 stacked DCRNN layers --------
        layers = []
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            layers.append(DCRNNCell(num_nodes, in_dim, hidden_dim, dropout))

        self.layers = nn.ModuleList(layers)

        # Final output projection
        self.fc_dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.register_buffer("adj", torch.FloatTensor(adj))

    def forward(self, x):
        B, T, N, F = x.shape
        h = [torch.zeros(B, N, self.hidden_dim, device=x.device)
             for _ in range(self.num_layers)]

        for t in range(T):
            input_t = x[:, t]   # (B, N, F)
            for l in range(self.num_layers):
                h[l] = self.layers[l](input_t, h[l], self.adj)
                input_t = h[l]   # output of one layer becomes input to next

        out = self.fc_dropout(h[-1])
        out = self.fc(out)
        return out.unsqueeze(1)




# ============================================================
# STEP 7 — TRAIN DCRNN
# ============================================================
# ============================================================
# STEP 7 — TRAIN DCRNN (Save best validation model)
# ============================================================
input_dim = 3
hidden_dim = 64
output_dim = 1

model = DCRNN(NUM_NODES, input_dim, hidden_dim, output_dim, adj_mx, dropout=0.2, num_layers=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

EPOCHS = 50
best_val_loss = float('inf')    # Track best loss

best_model_path = f"{data_dir}/dcrnn_325nodes_best.pth"

for epoch in range(EPOCHS):
    # ------------------------
    # TRAINING
    # ------------------------
    model.train()
    train_loss = 0

    for Xb, Yb in train_loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        optimizer.zero_grad()

        preds = model(Xb)               # (B, 1, N, 1)
        target = Yb[:, -1:, :, 0:1]     # final step target

        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ------------------------
    # VALIDATION
    # ------------------------
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for Xb, Yb in val_loader:
            preds = model(Xb.to(device))
            target = Yb[:, -1:, :, 0:1].to(device)
            val_loss += criterion(preds, target).item()

    avg_train = train_loss / len(train_loader)
    avg_val   = val_loss   / len(val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}  Train={avg_train:.6f}  Val={avg_val:.6f}")

    # ------------------------
    # SAVE BEST MODEL
    # ------------------------
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print(f"🔥 Saved BEST model (Val loss = {best_val_loss:.6f}) at: {best_model_path}")

# ============================================================
# FINAL PRINT
# ============================================================
print(f"\nTraining done. Best model saved at:\n → {best_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Loaded shapes: (36481, 325, 3) (5211, 325, 3) (10424, 325, 3)
Train shape: (36481, 325, 3)
Val shape: (5211, 325, 3)
Test shape: (10424, 325, 3)
Adjacency shape: (325, 325)
Sequence shapes:
X_train: (36466, 12, 325, 3)
Y_train: (36466, 3, 325, 3)
Epoch 1/50  Train=0.366226  Val=0.365691
🔥 Saved BEST model (Val loss = 0.365691) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 2/50  Train=0.301338  Val=0.346346
🔥 Saved BEST model (Val loss = 0.346346) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 3/50  Train=0.280765  Val=0.318477
🔥 Saved BEST model (Val loss = 0.318477) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 4/50  Train=0.268759  Val=0.306476
🔥 Saved BEST model (Val loss = 0.306476) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 5/50  Train=0.2

KeyboardInterrupt: 

Multilayer DCRNN (3 layers, 128 units each, dropout=0.2)


In [ ]:
# ============================================================
# STEP 1 — Google Drive + Imports
# ============================================================
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# STEP 2 — Load Normalized Data
# ============================================================
data_dir = "/content/drive/MyDrive/traffic_data"

train_full = np.load(f"{data_dir}/train_norm_PEMS_BAY.npz")["data"]
val_full   = np.load(f"{data_dir}/val_norm_PEMS_BAY.npz")["data"]
test_full  = np.load(f"{data_dir}/test_norm_PEMS_BAY.npz")["data"]

print("Loaded shapes:", train_full.shape, val_full.shape, test_full.shape)

# Limit timesteps EXACTLY as requested
train_data = train_full[:36481]   # (1000, 325, 3)
val_data   = val_full[:5211]      # (100, 325, 3)
test_data  = test_full[:10424]     # (200, 325, 3)

print("Train shape:", train_data.shape)
print("Val shape:",   val_data.shape)
print("Test shape:",  test_data.shape)

# ============================================================
# STEP 3 — Load REAL 325×325 Adjacency Matrix
# ============================================================
adj_path = f"{data_dir}/adj_mx_bay.pkl/adj_mx_bay.pkl"

with open(adj_path, "rb") as f:
    sensor_ids, sensor_id_to_ind, adj_mx = pickle.load(f, encoding="latin1")

adj_mx = np.array(adj_mx).astype(np.float32)
NUM_NODES = adj_mx.shape[0]

print("Adjacency shape:", adj_mx.shape)   # (325, 325)


# ============================================================
# STEP 4 — Build Temporal Sequences
# ============================================================
def create_sequences(data, seq_len=12, pred_len=3):
    """
    data: (T, N, F)
    returns:
       X: (B, seq_len, N, F)
       Y: (B, pred_len, N, F)
    """
    T = data.shape[0]
    X, Y = [], []

    for i in range(T - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        Y.append(data[i+seq_len:i+seq_len+pred_len])

    X = np.array(X)
    Y = np.array(Y)
    return X, Y

# Build sequences
seq_len = 12
pred_len = 3

X_train, Y_train = create_sequences(train_data, seq_len, pred_len)
X_val, Y_val     = create_sequences(val_data, seq_len, pred_len)
X_test, Y_test   = create_sequences(test_data, seq_len, pred_len)

print("Sequence shapes:")
print("X_train:", X_train.shape)
print("Y_train:", Y_train.shape)


# ============================================================
# STEP 5 — DataLoaders
# ============================================================
def to_loader(X, Y, batch=32, shuffle=True):
    return DataLoader(
        TensorDataset(torch.FloatTensor(X), torch.FloatTensor(Y)),
        batch_size=batch,
        shuffle=shuffle
    )

train_loader = to_loader(X_train, Y_train)
val_loader   = to_loader(X_val, Y_val)
test_loader  = to_loader(X_test, Y_test)


# ============================================================
# STEP 6 — DCRNN MODEL
# ============================================================
# ============================================================
# STEP 6 — DCRNN MODEL (with dropout=0.2)
# ============================================================
# ============================================================
# STEP 6 — Multilayer DCRNN (5 layers, 128 units each, dropout=0.2)
# ============================================================
class DiffusionConv(nn.Module):
    def __init__(self, num_nodes, input_dim, output_dim, dropout=0.2):
        super().__init__()
        self.theta = nn.Parameter(torch.randn(input_dim, output_dim))
        nn.init.xavier_uniform_(self.theta)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, adj):
        # x: (B, N, F)
        out = torch.einsum("ij,bjf->bif", adj, x)        # diffusion step
        out = torch.einsum("bif,fo->bio", out, self.theta)
        out = self.dropout(out)                          # dropout
        return out


class DCRNNCell(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, dropout=0.2):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim

        self.diff = DiffusionConv(num_nodes, input_dim + hidden_dim, hidden_dim, dropout)
        self.gru = nn.GRUCell(hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, h_prev, adj):
        combined = torch.cat([x, h_prev], dim=-1)
        diff_out = self.diff(combined, adj)
        diff_out = self.dropout(diff_out)
        h_new = self.gru(diff_out.reshape(-1, self.hidden_dim),
                         h_prev.reshape(-1, self.hidden_dim))
        return h_new.reshape(-1, self.num_nodes, self.hidden_dim)


class DCRNN(nn.Module):
    def __init__(self, num_nodes, input_dim, hidden_dim, output_dim, adj, dropout=0.2, num_layers=5):
        super().__init__()
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # -------- Create 5 stacked DCRNN layers --------
        layers = []
        for i in range(num_layers):
            in_dim = input_dim if i == 0 else hidden_dim
            layers.append(DCRNNCell(num_nodes, in_dim, hidden_dim, dropout))

        self.layers = nn.ModuleList(layers)

        # Final output projection
        self.fc_dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.register_buffer("adj", torch.FloatTensor(adj))

    def forward(self, x):
        B, T, N, F = x.shape
        h = [torch.zeros(B, N, self.hidden_dim, device=x.device)
             for _ in range(self.num_layers)]

        for t in range(T):
            input_t = x[:, t]   # (B, N, F)
            for l in range(self.num_layers):
                h[l] = self.layers[l](input_t, h[l], self.adj)
                input_t = h[l]   # output of one layer becomes input to next

        out = self.fc_dropout(h[-1])
        out = self.fc(out)
        return out.unsqueeze(1)




# ============================================================
# STEP 7 — TRAIN DCRNN
# ============================================================
# ============================================================
# STEP 7 — TRAIN DCRNN (Save best validation model)
# ============================================================
input_dim = 3
hidden_dim = 128
output_dim = 1

model = DCRNN(NUM_NODES, input_dim, hidden_dim, output_dim, adj_mx, dropout=0.2, num_layers=3).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

EPOCHS = 50
best_val_loss = float('inf')    # Track best loss

best_model_path = f"{data_dir}/dcrnn_325nodes_best.pth"

for epoch in range(EPOCHS):
    # ------------------------
    # TRAINING
    # ------------------------
    model.train()
    train_loss = 0

    for Xb, Yb in train_loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        optimizer.zero_grad()

        preds = model(Xb)               # (B, 1, N, 1)
        target = Yb[:, -1:, :, 0:1]     # final step target

        loss = criterion(preds, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # ------------------------
    # VALIDATION
    # ------------------------
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for Xb, Yb in val_loader:
            preds = model(Xb.to(device))
            target = Yb[:, -1:, :, 0:1].to(device)
            val_loss += criterion(preds, target).item()

    avg_train = train_loss / len(train_loader)
    avg_val   = val_loss   / len(val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}  Train={avg_train:.6f}  Val={avg_val:.6f}")

    # ------------------------
    # SAVE BEST MODEL
    # ------------------------
    if avg_val < best_val_loss:
        best_val_loss = avg_val
        torch.save(model.state_dict(), best_model_path)
        print(f"🔥 Saved BEST model (Val loss = {best_val_loss:.6f}) at: {best_model_path}")

# ============================================================
# FINAL PRINT
# ============================================================
print(f"\nTraining done. Best model saved at:\n → {best_model_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Loaded shapes: (36481, 325, 3) (5211, 325, 3) (10424, 325, 3)
Train shape: (36481, 325, 3)
Val shape: (5211, 325, 3)
Test shape: (10424, 325, 3)
Adjacency shape: (325, 325)
Sequence shapes:
X_train: (36466, 12, 325, 3)
Y_train: (36466, 3, 325, 3)
Epoch 1/50  Train=0.362205  Val=0.357451
🔥 Saved BEST model (Val loss = 0.357451) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 2/50  Train=0.291393  Val=0.326213
🔥 Saved BEST model (Val loss = 0.326213) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 3/50  Train=0.271285  Val=0.313607
🔥 Saved BEST model (Val loss = 0.313607) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 4/50  Train=0.260252  Val=0.307929
🔥 Saved BEST model (Val loss = 0.307929) at: /content/drive/MyDrive/traffic_data/dcrnn_325nodes_best.pth
Epoch 5/50  Train=0.2

KeyboardInterrupt: 